<img src="http://eikon.tpq.io/refinitiv_logo.png" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">

# Eikon Data API

**Cross-Asset Financial Analytics &mdash; The Random Walk Hypothesis Revisited**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial shows

* how to retrieve historical data across asset classes via the Eikon Data API,
* how to work with such data using `pandas`, `Plotly` and `Cufflinks` and
* how to derive support for the Random Walk Hypothesis from financial time series data.

## Random Walk Hypothesis

Eugene F. Fama (1965): “Random Walks in Stock Market Prices”:

> “For many years, economists, statisticians, and teachers of finance have been interested in developing and testing models of stock price behavior. One important model that has evolved from this research is the theory of random walks. This theory casts serious doubt on many other methods for describing and predicting stock price behavior—methods that have considerable popularity outside the academic world. For example, we shall see later that, if the random-walk theory is an accurate description of reality, then the various “technical” or “chartist” procedures for predicting stock prices are completely without value.”

Michael Jensen (1978): “Some Anomalous Evidence Regarding Market Efficiency”:

>“A market is efficient with respect to an information set S if it is impossible to make economic profits by trading on the basis of information set S.”

If a stock price follows a (simple) random walk (no drift & normally distributed returns), then it rises and falls with the same probability of 50% (“toss of a coin”).

**In such a case, the best predictor of tomorrow’s stock price — in a least-squares sense — is today’s stock price.**

## Importing Required Packages

In [8]:
import eikon as ek  # the Eikon Python wrapper package
ek.set_app_key('0ed6a35e0937415eab446d3375bca7cf671d6b4c')
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp

The following **Python and package versions** are used.

In [2]:
import sys
print(sys.version)

3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]


In [3]:
ek.__version__

'0.1.10'

In [4]:
np.__version__

'1.13.3'

In [5]:
pd.__version__

'0.22.0'

In [6]:
cf.__version__

'0.12.1'

In [113]:
import cx_oracle as ol
cx_Oracle.__version__

ModuleNotFoundError: No module named 'cx_oracle'

## Connecting to Eikon Data API

This code sets the `app_id` to connect to the **Eikon Data API Proxy** which needs to be running locally.

In [3]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')

[]

## Retrieving Cross-Asset Data

We first define a **small universe of `RICS`** for which to retrieve data.

In [9]:
rics = [
    'GE',  # General Electric stock
    'AAPL.O',  # Apple stock
    '.SPX',  # S&P 500 stock index
    '.VIX',  # VIX volatility index
    'EUR=',  # EUR/USD exchange rate
    'XAU=',  # Gold price
    'DE10YT=RR',  # 10yr Bund price
]

Second, **end-of-day (EOD) data** is retrieved.

In [10]:
data = ek.get_timeseries(rics, fields='CLOSE',
                         start_date='2019-01-01',
                         end_date='2019-12-31')

In [12]:
data.head()  # first five rows

CLOSE,GE,AAPL.O,.SPX,.VIX,EUR=,XAU=,DE10YT=RR
Date,,,,,,,
2019-01-01,NaN,NaN,NaN,NaN,1.1462,1282.17,NaN
2019-01-02,7.738071,157.92,2510.03,23.22,1.1342,1284.74,0.171
2019-01-03,7.747683,142.19,2447.89,25.45,1.1392,1293.61,0.151
2019-01-04,7.911096,148.26,2531.94,21.38,1.1393,1284.83,0.208
2019-01-07,8.401334,147.93,2549.69,21.40,1.1474,1288.63,0.216


In [7]:
data.tail()  # final five rows

CLOSE,GE,AAPL.O,.SPX,.VIX,EUR=,XAU=,DE10YT=RR
Date,,,,,,,
2017-12-25,NaN,NaN,NaN,NaN,1.1868,1274.86,NaN
2017-12-26,16.754605,170.57,2680.50,10.25,1.1858,1283.24,NaN
2017-12-27,16.706542,170.60,2682.62,10.47,1.1886,1287.01,0.382
2017-12-28,16.687317,171.08,2687.54,10.18,1.1941,1294.75,0.421
2017-12-29,16.773830,169.23,2673.61,11.04,1.1996,1302.45,0.424


Only complete data rows are selected.

In [13]:
data.dropna(inplace=True)  # deletes tows with NaN values

In [14]:
data.info()  # DataFrame meta information

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 248 entries, 2019-01-02 to 2019-12-31
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   GE         248 non-null    float64
 1   AAPL.O     248 non-null    float64
 2   .SPX       248 non-null    float64
 3   .VIX       248 non-null    float64
 4   EUR=       248 non-null    float64
 5   XAU=       248 non-null    float64
 6   DE10YT=RR  248 non-null    float64
dtypes: float64(7)
memory usage: 15.5 KB


## Calculating the Log Returns

We next calculate the **log returns** in vectorized fashion.

In [15]:
rets = np.log(data / data.shift(1))  # log returns in vectorized fashion

In [11]:
rets.head()

CLOSE,GE,AAPL.O,.SPX,.VIX,EUR=,XAU=,DE10YT=RR
Date,,,,,,,
2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,0.000316,-0.001120,0.005706,-0.081016,0.007851,0.003945,0.025689
2017-01-05,-0.005694,0.005072,-0.000771,-0.015306,0.011096,0.014498,-0.075223
2017-01-06,0.002851,0.011087,0.003511,-0.030450,-0.006909,-0.006519,0.178408
2017-01-09,-0.004757,0.009118,-0.003555,0.020980,0.003981,0.007171,-0.071096


`pandas` allows to derive the **correlation matrix** with a single method call.

In [16]:
data.corr()  # correlation matrix by column

CLOSE,GE,AAPL.O,.SPX,.VIX,EUR=,XAU=,DE10YT=RR
CLOSE,,,,,,,
GE,1.000000,0.513866,0.597670,-0.639145,-0.083479,0.027055,0.057881
AAPL.O,0.513866,1.000000,0.942667,-0.493353,-0.713562,0.717054,-0.563630
.SPX,0.597670,0.942667,1.000000,-0.619961,-0.691158,0.679387,-0.607516
.VIX,-0.639145,-0.493353,-0.619961,1.000000,0.169215,-0.061255,-0.025260
EUR=,-0.083479,-0.713562,-0.691158,0.169215,1.000000,-0.721606,0.785640
XAU=,0.027055,0.717054,0.679387,-0.061255,-0.721606,1.000000,-0.884749
DE10YT=RR,0.057881,-0.563630,-0.607516,-0.025260,0.785640,-0.884749,1.000000


## Plotting the Data

Using `Cufflinks`, we can plot the normalized financial time series as **line plots** for comparison.

In [17]:
cf.set_config_file(offline=True)  # set the plotting mode to offline

In [18]:
data.normalize().iplot(kind='lines')

The frequeny distributions, i.e. the **histograms**, of the log returns per `RIC`.

In [19]:
rets.iplot(kind='histogram', subplots=True)

The **heatmap** below visualizes the correlations between the financial time series.

In [20]:
data.corr().iplot(kind='heatmap', colorscale='blues')

## Preparing Lagged Data

To gain insights into whether the random walk hypothesis holds true, we work with **five lags**. The code that follows derives the **lagged data** for every single `RIC`. First, a function that adds columns with lagged data to a `DataFrame` object.

In [21]:
def add_lags(data, ric, lags):
    cols = []
    df = pd.DataFrame(data[ric])
    for lag in range(1, lags + 1):
        col = 'lag_{}'.format(lag)  # defines the column name
        df[col] = df[ric].shift(lag)  # creates the lagged data column
        cols.append(col)  # stores the column name
    df.dropna(inplace=True)  # gets rid of incomplete data rows
    return df, cols

Second, the iterations over all `RICs`, using the `add_lags` function and storing the resulting `DataFrame` objects in a dictonary.

In [22]:
lags = 5  # five historical lags
df,T = add_lags(data,'GE',lags)

In [23]:
type(data['GE'])

pandas.core.series.Series

In [25]:
df['GE']

Date
2019-01-09     8.170633
2019-01-10     8.593584
2019-01-11     8.593584
2019-01-14     8.555134
2019-01-15     8.391721
                ...    
2019-12-20    11.030000
2019-12-23    11.160000
2019-12-27    11.180000
2019-12-30    11.080000
2019-12-31    11.160000
Name: GE, Length: 243, dtype: float64

In [26]:
dfs = {}
for ric in rics:
    df, cols = add_lags(data, ric, lags)
    dfs[ric] = df

In [27]:
cols  # the column names for the lags


['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']

In [28]:
dfs.keys()  # the keys of the dictonary


dict_keys(['GE', 'AAPL.O', '.SPX', '.VIX', 'EUR=', 'XAU=', 'DE10YT=RR'])

In [29]:
dfs['AAPL.O'].head(7)

,AAPL.O,lag_1,lag_2,lag_3,lag_4,lag_5
Date,,,,,,
2019-01-09,153.31,150.75,147.93,148.26,142.19,157.92
2019-01-10,153.80,153.31,150.75,147.93,148.26,142.19
2019-01-11,152.29,153.80,153.31,150.75,147.93,148.26
2019-01-14,150.00,152.29,153.80,153.31,150.75,147.93
2019-01-15,153.07,150.00,152.29,153.80,153.31,150.75
2019-01-16,154.94,153.07,150.00,152.29,153.80,153.31
2019-01-17,155.86,154.94,153.07,150.00,152.29,153.80


## Implementing OLS Regression

The matrix consisting of the lagged data columns is used to "predict" the next day's value of the `RIC` via **linear OLS regression**.

In [30]:
regs = {}
for ric in rics:
    df = dfs[ric]  # getting data for the RIC
    reg = np.linalg.lstsq(df[cols], df[ric], rcond=-1)[0]  # the OLS regression
    regs[ric] = reg  # storing the results

In [60]:
ric ='AAPL.O'
regs = {}
df = dfs[ric]
reg = np.linalg.lstsq(df[cols], df[ric], rcond=-1)
regs[ric] = reg
reg

(array([ 1.00717172, -0.14691012,  0.11575033,  0.05903115, -0.03179905]),
 array([2198.42872905]),
 5,
 array([7306.96512541,   71.09980283,   43.91878063,   31.39174015,
          23.85514461]))

In [56]:
cols

['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']

In [35]:
for ric in rics:
    print('{:10} | {}'.format(ric, regs[ric]))

GE         | [ 0.96817868  0.01936998  0.08795867  0.03609661 -0.11092979]
AAPL.O     | [ 1.00717172 -0.14691012  0.11575033  0.05903115 -0.03179905]
.SPX       | [ 0.93289039  0.00308899  0.07863694 -0.01590131  0.00229603]
.VIX       | [0.80268564 0.12030209 0.00499019 0.00914989 0.0575344 ]
EUR=       | [ 0.97259488  0.09903832 -0.02550018 -0.20460307  0.15837288]
XAU=       | [ 0.97517376  0.02257361 -0.00480906 -0.00624851  0.01399871]
DE10YT=RR  | [ 0.93916618  0.12588257 -0.03018844  0.03820625 -0.07749446]


## Taking a Closer Look

Let's pick one `RIC` and compare the original time series with the OLS predicted one.

In [36]:
ric = 'AAPL.O'
dfs[ric]

,AAPL.O,lag_1,lag_2,lag_3,lag_4,lag_5
Date,,,,,,
2019-01-09,153.31,150.75,147.93,148.26,142.19,157.92
2019-01-10,153.80,153.31,150.75,147.93,148.26,142.19
2019-01-11,152.29,153.80,153.31,150.75,147.93,148.26
2019-01-14,150.00,152.29,153.80,153.31,150.75,147.93
2019-01-15,153.07,150.00,152.29,153.80,153.31,150.75
...,...,...,...,...,...,...
2019-12-20,279.44,280.02,279.74,280.41,279.86,275.15
2019-12-23,284.00,279.44,280.02,279.74,280.41,279.86
2019-12-27,289.80,284.00,279.44,280.02,279.74,280.41


In [63]:
dfs

{'GE':                    GE      lag_1      lag_2      lag_3      lag_4      lag_5
 Date                                                                        
 2019-01-09   8.170633   8.228309   8.401334   7.911096   7.747683   7.738071
 2019-01-10   8.593584   8.170633   8.228309   8.401334   7.911096   7.747683
 2019-01-11   8.593584   8.593584   8.170633   8.228309   8.401334   7.911096
 2019-01-14   8.555134   8.593584   8.593584   8.170633   8.228309   8.401334
 2019-01-15   8.391721   8.555134   8.593584   8.593584   8.170633   8.228309
 ...               ...        ...        ...        ...        ...        ...
 2019-12-20  11.030000  11.040000  10.960000  11.100000  11.170000  11.340000
 2019-12-23  11.160000  11.030000  11.040000  10.960000  11.100000  11.170000
 2019-12-27  11.180000  11.160000  11.030000  11.040000  10.960000  11.100000
 2019-12-30  11.080000  11.180000  11.160000  11.030000  11.040000  10.960000
 2019-12-31  11.160000  11.080000  11.180000  11.160000  1

In [62]:
print(dfs[ric][ric])
res = pd.DataFrame(dfs[ric][ric])  # picks the original time series

Date
2019-01-09    153.31
2019-01-10    153.80
2019-01-11    152.29
2019-01-14    150.00
2019-01-15    153.07
               ...  
2019-12-20    279.44
2019-12-23    284.00
2019-12-27    289.80
2019-12-30    291.52
2019-12-31    293.65
Name: AAPL.O, Length: 243, dtype: float64


In [38]:
dfs['GE'][cols]

,lag_1,lag_2,lag_3,lag_4,lag_5
Date,,,,,
2019-01-09,8.228309,8.401334,7.911096,7.747683,7.738071
2019-01-10,8.170633,8.228309,8.401334,7.911096,7.747683
2019-01-11,8.593584,8.170633,8.228309,8.401334,7.911096
2019-01-14,8.593584,8.593584,8.170633,8.228309,8.401334
2019-01-15,8.555134,8.593584,8.593584,8.170633,8.228309
...,...,...,...,...,...
2019-12-20,11.040000,10.960000,11.100000,11.170000,11.340000
2019-12-23,11.030000,11.040000,10.960000,11.100000,11.170000
2019-12-27,11.160000,11.030000,11.040000,10.960000,11.100000


In [39]:
res['PRED'] = np.dot(dfs[ric][cols], regs[ric])  # creates the "prediction" values

The **predicted prices** are almost exactly the prices from the day before.

In [40]:
res.iloc[-50:].iplot()

In [41]:
res.head()

,AAPL.O,PRED
Date,,
2019-01-09,153.31,150.631799
2019-01-10,153.80,153.616193
2019-01-11,152.29,153.847533
2019-01-14,150.00,152.728000
2019-01-15,153.07,150.761575


## Analyzing the Results

Now analyzing the **regression results** a bit more formally.

In [42]:
rega = np.stack(regs.values())  # combines the regression results
rega

array([[ 0.96817868,  0.01936998,  0.08795867,  0.03609661, -0.11092979],
       [ 1.00717172, -0.14691012,  0.11575033,  0.05903115, -0.03179905],
       [ 0.93289039,  0.00308899,  0.07863694, -0.01590131,  0.00229603],
       [ 0.80268564,  0.12030209,  0.00499019,  0.00914989,  0.0575344 ],
       [ 0.97259488,  0.09903832, -0.02550018, -0.20460307,  0.15837288],
       [ 0.97517376,  0.02257361, -0.00480906, -0.00624851,  0.01399871],
       [ 0.93916618,  0.12588257, -0.03018844,  0.03820625, -0.07749446]])

Almost all the weight lies on the most recent price (`lag_1`).

In [43]:
rega.mean(axis=0)  # mean values by column

array([ 0.94255161,  0.03476363,  0.03240549, -0.01203843,  0.00171125])

In [44]:
regd = pd.DataFrame(rega, columns=cols, index=rics)  # converting the results to DataFrame

In [45]:
regd

,lag_1,lag_2,lag_3,lag_4,lag_5
GE,0.968179,0.019370,0.087959,0.036097,-0.110930
AAPL.O,1.007172,-0.146910,0.115750,0.059031,-0.031799
.SPX,0.932890,0.003089,0.078637,-0.015901,0.002296
.VIX,0.802686,0.120302,0.004990,0.009150,0.057534
EUR=,0.972595,0.099038,-0.025500,-0.204603,0.158373
XAU=,0.975174,0.022574,-0.004809,-0.006249,0.013999
DE10YT=RR,0.939166,0.125883,-0.030188,0.038206,-0.077494


In [46]:
regd.describe()  # summary statistics

,lag_1,lag_2,lag_3,lag_4,lag_5
count,7.000000,7.000000,7.000000,7.000000,7.000000
mean,0.942552,0.034764,0.032405,-0.012038,0.001711
std,0.066416,0.094996,0.059969,0.088948,0.089345
min,0.802686,-0.146910,-0.030188,-0.204603,-0.110930
25%,0.936028,0.011229,-0.015155,-0.011075,-0.054647
50%,0.968179,0.022574,0.004990,0.009150,0.002296
75%,0.973884,0.109670,0.083298,0.037151,0.035767
max,1.007172,0.125883,0.115750,0.059031,0.158373


## Visualizing the Results

The following bar chart illustrates that the results a qualitatively similar for all `RICS` analyzed &mdash; "_today's price is the best predictor, in a least-squares sense, for tomorrow's price_".

In [48]:
regd.iplot(kind='bar')

In [49]:
regd

,lag_1,lag_2,lag_3,lag_4,lag_5
GE,0.968179,0.019370,0.087959,0.036097,-0.110930
AAPL.O,1.007172,-0.146910,0.115750,0.059031,-0.031799
.SPX,0.932890,0.003089,0.078637,-0.015901,0.002296
.VIX,0.802686,0.120302,0.004990,0.009150,0.057534
EUR=,0.972595,0.099038,-0.025500,-0.204603,0.158373
XAU=,0.975174,0.022574,-0.004809,-0.006249,0.013999
DE10YT=RR,0.939166,0.125883,-0.030188,0.038206,-0.077494


The **mean values** for the single optimal regression parameters.

In [93]:
regd.mean().iplot(kind='bar')

## Analyzing Intraday Data

Let us quickly check, whether the results are similar on an **intraday basis**.

In [96]:
data = ek.get_timeseries(rics,  # RICs
              fields='CLOSE',  # fields to be retrieved
              start_date='2020-01-08 14:00:00',  # start time
              end_date='2020-01-08 18:00:00',  # end time
              interval='minute')  # bar length

In [97]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 241 entries, 2020-01-08 14:00:00 to 2020-01-08 18:00:00
Freq: T
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   GE         241 non-null    float64
 1   AAPL.O     241 non-null    float64
 2   .SPX       210 non-null    float64
 3   .VIX       225 non-null    float64
 4   EUR=       241 non-null    float64
 5   XAU=       241 non-null    float64
 6   DE10YT=RR  241 non-null    float64
dtypes: float64(7)
memory usage: 15.1 KB


In [101]:
data.tail()

CLOSE,GE,AAPL.O,.SPX,.VIX,EUR=,XAU=,DE10YT=RR
Date,,,,,,,
2020-01-08 17:56:00,11.9850,302.5951,3258.03,13.24,1.1116,1561.6765,-0.241
2020-01-08 17:57:00,11.9800,302.6390,3258.41,13.23,1.1114,1561.2200,-0.241
2020-01-08 17:58:00,11.9950,302.6776,3259.05,13.20,1.1116,1561.2950,-0.242
2020-01-08 17:59:00,11.9942,302.7400,3259.12,13.18,1.1116,1561.4904,-0.241
2020-01-08 18:00:00,11.9900,302.6400,3258.41,13.21,1.1116,1561.6475,-0.241


In [102]:
dfs = {}
for ric in rics:
    df, cols = add_lags(data, ric, lags)
    dfs[ric] = df

In [103]:
regs = {}
for ric in rics:
    df = dfs[ric]
    reg = np.linalg.lstsq(df[cols], df[ric], rcond=-1)[0]
    regs[ric] = reg

In [104]:
rega = np.stack(regs.values())

In [105]:
regd = pd.DataFrame(rega, columns=cols, index=rics)

**Intraday** the optimal regression parameters show more variation.

In [106]:
regd.iplot(kind='bar')

In [107]:
regd.mean().iplot(kind='bar')

## Conclusions

Based on this tutorial, we can conclude that

* it is easy to retrieve **historical end-of-day and intraday data across asset classes** via the Eikon Data API,
* `Plotly` and `Cufflinks` make **financial data visualization** convenient and
* there is **support for the Random Walk Hypothesis** based on the OLS regression analysis (both daily and a little bit less so intraday).

## Eikon Data API Developer Resources

* [Overview](https://developers.thomsonreuters.com/eikon-data-apis) 
* [Quick Start ](https://developers.thomsonreuters.com/eikon-data-apis/quick-start)
* [Documentation](https://developers.thomsonreuters.com/eikon-data-apis/docs)
* [Downloads](https://developers.thomsonreuters.com/eikon-data-apis/downloads)
* [Tutorials](https://developers.thomsonreuters.com/eikon-data-apis/learning)
* [Q&A Forums](https://developers.thomsonreuters.com/eikon-data-apis/qa) 

Data Item Browser Application: Type `DIB` into Eikon Search Bar.

<img src="http://eikon.tpq.io/refinitiv_logo.png" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">